## Loading Dataset

In [6]:
from datasets import load_dataset

hf_dataset = load_dataset(
    "hotpot_qa", 
    "distractor",
    split="validation[:100]",
    cache_dir="/mnt/d/datasets/hotpot_qa"
)

hf_dataset = hf_dataset.select(range(25))

In [9]:
import os
from agents.prompt_and_workflow_orchestration.orchestration import OrchestrationAgent
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

EXPERIMENT_NAME = "hotpot_qa_orchestration_2"

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
generated_data_path = os.path.join(project_root, 'data', 'generated', f'{EXPERIMENT_NAME}.parquet')


# planner_llm = ChatOllama(model="qwen3:8b", temperature=0.6)

planner_llm = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    model="x-ai/grok-4-fast:free",
    temperature=0.6
)

high_temp_llm= ChatOllama(model="mistral:7b", temperature=0.8)
medium_temp_llm= ChatOllama(model="mistral:7b", temperature=0.5)
low_temp_llm= ChatOllama(model="mistral:7b", temperature=0.2)

agent = OrchestrationAgent(
    planner_llm=planner_llm,
    high_temp_llm=high_temp_llm,
    medium_temp_llm=medium_temp_llm,
    low_temp_llm=low_temp_llm,
)

## Evaluating the `QueryEngine`


### Preparing dataset

In [10]:
from contextlib import redirect_stdout
from io import StringIO

ragas_dataset = []

for i, item in enumerate(hf_dataset):
    print(f"{i+1} iteration")
    question = item["question"]
    answer = item["answer"]
    context = item["context"]["sentences"]

    # Silence the agent's output
    with redirect_stdout(StringIO()):
        response = agent.generate_response(question, context)
        
    ragas_dataset.append(
        {
            "user_input": question,
            "retrieved_contexts": [str(item) for item in context],
            "response": response["content"].strip(),
            "reference": answer,
            "workflow_plan": response["workflow_plan"]
        }
    )


df = pd.DataFrame(ragas_dataset)
df.to_parquet(generated_data_path, index=False)

1 iteration
2 iteration
3 iteration
4 iteration
5 iteration
6 iteration
7 iteration
8 iteration
9 iteration
10 iteration
11 iteration
12 iteration
13 iteration
14 iteration
15 iteration
16 iteration
17 iteration
18 iteration
19 iteration
20 iteration
21 iteration
22 iteration
23 iteration
24 iteration
25 iteration


JSONDecodeError: Expecting value: line 143 column 1 (char 781)